In [1]:
# from datasets import load_dataset
# from training_datasets.GLUEDataset import GLUEDataset

# cola = load_dataset('glue','cola')
# # print(cola)
# # sst2 = load_dataset('glue','sst2')
# # print(sst2)
# mrpc = load_dataset('glue', 'mrpc')
# # print(mrpc)
# pt_cola = GLUEDataset([cola, mrpc], 'train')

In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

meta_ds = GLUEMetaDataset(k=4,numTasks=1000)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 339.66it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 38.59it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████

In [3]:
train_protomaml_sampler = FewShotEpisodeBatchSampler(meta_ds, kShot=4, batchSize=4)

train_protomaml_loader = data.DataLoader(
    meta_ds, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [4]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [5]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [6]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

# for i in range(1):
#     batch = next(iter(val_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [7]:
from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

In [ ]:
from training.trainer import train_model

protomaml_model = train_model(
    ProtoFOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=4, warmupSteps=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42

  | Name        | Type               | Params
---------------------------------------------------
0 | metaLearner | PrototypeMetaModel | 108 M 
---------------------------------------------------
7.3 M     Trainable params
101 M     Non-trainable params
108 M     Total params
434.029   Total estimated model params size (MB)
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Number of labels in the episode are 2 and lines are 1
inner loop training loss is 1.2988698482513428
inner loop training loss is 1.2566895484924316
inner loop training loss is 1.216719627380371
inner loop training loss is 1.1787711381912231
inner loop training loss is 1.1426889896392822
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 3.7598679065704346
inner loop training loss is 3.676438331604004
inner loop training loss is 3.6006839275360107
inner loop training loss is 3.5310091972351074
inner loop training loss is 3.4662742614746094
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 3.4543853998184204
inner loop training loss is 3.3849124908447266
inner loop training loss is 3.319882869720459
inner loop training loss is 3.2585484981536865
inner loop training loss is 3.2003565430641174
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 3.738795757293701
inner loop training loss is 3.676833

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/loops/optimization/automatic.py:129: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Number of labels in the episode are 2 and lines are 1
inner loop training loss is 1.2469303607940674
inner loop training loss is 1.2074127197265625
inner loop training loss is 1.16989266872406
inner loop training loss is 1.1342215538024902
inner loop training loss is 1.1002651453018188
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 3.5955933332443237
inner loop training loss is 3.51630437374115
inner loop training loss is 3.444008469581604
inner loop training loss is 3.3772873878479004
inner loop training loss is 3.315114378929138
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 3.200300455093384
inner loop training loss is 3.1354746222496033
inner loop training loss is 3.0746588706970215
inner loop training loss is 3.017196297645569
inner loop training loss is 2.962587893009186
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 3.4860527515411377
inner loop training loss is 3.42685210704

inner loop training loss is 3.9011387825012207
inner loop training loss is 4.169217467308044
inner loop training loss is 3.828054428100586
outer loop training loss is 6.151514768600464
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 10.667069673538208
inner loop training loss is 10.606245040893555
inner loop training loss is 11.573100805282593
inner loop training loss is 10.444779872894287
inner loop training loss is 10.387210845947266
outer loop training loss is 14.1695556640625
Number of labels in the episode are 2 and lines are 1
inner loop training loss is 4.283499002456665
inner loop training loss is 4.101740717887878
inner loop training loss is 3.360090136528015
inner loop training loss is 3.5504798889160156
inner loop training loss is 3.091010332107544
outer loop training loss is 4.92769718170166
Number of labels in the episode are 2 and lines are 1
inner loop training loss is 4.689795017242432
inner loop training loss is 4.995008230209351
inner